In [32]:
pip install folium

     ---------------------------------------- 97.0/97.0 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 56.3/56.3 kB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [39]:
pip install geocoder

     ---------------------------------------- 98.6/98.6 kB 1.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# Import Libraries

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [33]:
import folium
from folium.plugins import MarkerCluster

# Load data

In [2]:
np.random.seed(42)
num_samples = 1000

In [3]:
df=pd.read_csv('Path sample gen.xlsx - Vehicle Routes.csv')

In [4]:
df

,Index,Route ID,Target Time,Time to Next Stop,Stop,Postcode
0,1,RTE01,17:15,0:15,SGH Pathology Lab,SO16 6YD
1,2,RTE01,17:30,0:20,Surgery 61,SO40 3JD
2,3,RTE01,17:50,0:10,Surgery 1,SO51 8XA
3,4,RTE01,18:00,0:10,Surgery 5,SO51 8JH
4,5,RTE01,18:10,0:15,Surgery 63,SO51 7LB
...,...,...,...,...,...,...
179,180,RTE10,12:40,0:20,Surgery 36,SO43 7FJ
180,181,RTE10,13:00,0:05,Surgery 27,SO45 4JG
181,182,RTE10,13:05,0:05,Surgery 72,SO45 5GQ
182,183,RTE10,13:10,0:30,Surgery 71,SO45 6AG


In [5]:
df.head()

,Index,Route ID,Target Time,Time to Next Stop,Stop,Postcode
0,1,RTE01,17:15,0:15,SGH Pathology Lab,SO16 6YD
1,2,RTE01,17:30,0:20,Surgery 61,SO40 3JD
2,3,RTE01,17:50,0:10,Surgery 1,SO51 8XA
3,4,RTE01,18:00,0:10,Surgery 5,SO51 8JH
4,5,RTE01,18:10,0:15,Surgery 63,SO51 7LB


In [6]:
df.tail()

,Index,Route ID,Target Time,Time to Next Stop,Stop,Postcode
179,180,RTE10,12:40,0:20,Surgery 36,SO43 7FJ
180,181,RTE10,13:00,0:05,Surgery 27,SO45 4JG
181,182,RTE10,13:05,0:05,Surgery 72,SO45 5GQ
182,183,RTE10,13:10,0:30,Surgery 71,SO45 6AG
183,184,RTE10,13:40,NaN,SGH Pathology Lab,SO16 6YD


In [7]:
df.describe()

,Index
count,184.000000
mean,92.500000
std,53.260367
min,1.000000
25%,46.750000
50%,92.500000
75%,138.250000
max,184.000000


In [9]:
df.shape

(184, 6)

In [11]:
df.info

<bound method DataFrame.info of      Index Route ID Target Time Time to Next Stop               Stop  Postcode
0        1    RTE01       17:15              0:15  SGH Pathology Lab  SO16 6YD
1        2    RTE01       17:30              0:20         Surgery 61  SO40 3JD
2        3    RTE01       17:50              0:10          Surgery 1  SO51 8XA
3        4    RTE01       18:00              0:10          Surgery 5  SO51 8JH
4        5    RTE01       18:10              0:15         Surgery 63  SO51 7LB
..     ...      ...         ...               ...                ...       ...
179    180    RTE10       12:40              0:20         Surgery 36  SO43 7FJ
180    181    RTE10       13:00              0:05         Surgery 27  SO45 4JG
181    182    RTE10       13:05              0:05         Surgery 72  SO45 5GQ
182    183    RTE10       13:10              0:30         Surgery 71  SO45 6AG
183    184    RTE10       13:40               NaN  SGH Pathology Lab  SO16 6YD

[184 rows x 6 colum

# Preprocess data

In [17]:
df['Target Time'] = df['Target Time'].str.replace(':','')

In [18]:
#df['time'] = df['Target Time'].apply(lambda x: x[:2]).astype(int) 
df['time'] = df['Target Time'].apply(lambda x: x[:2])

In [19]:
df['stop_type'] = df['Stop'].apply(lambda x: 0 if 'Surgery' in x else 1) # Label stop type 

In [21]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['stop_id'] = le.fit_transform(df['Stop']) 

In [22]:
X = df[['time', 'stop_type', 'stop_id']]
y = df['Route ID']

# Split data

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1) 

# Train model

In [25]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

# Evaluate model

In [26]:
print("Train Accuracy: ", model.score(X_train, y_train))  

Train Accuracy:  0.9183673469387755


In [27]:
print("Test Accuracy: ", model.score(X_test, y_test))

Test Accuracy:  0.1891891891891892


# Make prediction

In [48]:
new_data = [[13, 0, 23]] 

In [49]:
prediction = model.predict(new_data)

E:\Annaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [54]:
print("Predicted Route: ", prediction[0])

Predicted Route:  RTE10


In [60]:
import pandas as pd
import folium
import random
import geocoder 
from folium.plugins import MarkerCluster

In [56]:
df['lat'] = df['Postcode'].apply(lambda x: geocoder.osm(x).lat)
df['lon'] = df['Postcode'].apply(lambda x: geocoder.osm(x).lng)

In [57]:
m = folium.Map(location=[51.509865, -0.118092], zoom_start=10)

In [58]:
routes = df['Route ID'].unique()

In [61]:
colors = ['red', 'blue', 'green', 'purple', 'orange', 'gray', 'pink']
color_map = {route: random.choice(colors) for route in routes}

In [62]:
for route in routes:

    route_df = df[df['Route ID'] == route]  
    
    lats = list(route_df['lat'])
    lons = list(route_df['lon'])

    folium.PolyLine(locations=[list(t) for t in zip(lats, lons)],
                    color=color_map[route],
                    weight=4,
                    opacity=1).add_to(m) 

    lats = list(route_df['lat'])
    lons = list(route_df['lon'])
    MarkerCluster(locations=[list(t) for t in zip(lats, lons)],
                  popups=list(route_df['Stop']), 
                  show=True).add_to(m)

m